In [1]:
import pycrfsuite
import pandas as pd
from address_compare.parsers import hyphen_parse
import json
from address_compare.feature_functions import WordFeatures1 as wf
from collections import OrderedDict
from numpy.random import uniform, seed
from importlib import reload
from address_compare.constants import DIRECTIONS, STREET_TYPES, UNIT_TYPES
from string import punctuation
from address_compare.feature_functions import FeatureFunctionApplicator
import numpy as np
import re

# Training a Conditional Random Field-based address parser

In this notebook I will show how we trained a CRF model to parse addresses into their component parts.

## Introduction

How do we, as humans, know when two representations of an address are pointing to the same address? The strings "1401-88 W. HASTINGS ST" and "88 West Hastings Street, Ste. 1401" are very different, but refer to the same physical place. But despite the large differences in the string representations of this location, most people with at least a little bit of experience with addresses can quickly spot that they are the same. How?

One theory that we have is that comparisons are made component-wise. We can see that each string representation points to an address with unit number 1401 and street number 88. We also know that street names are in general case-insensitive, so "HASTINGS" and "Hastings" are _probably_ the same street name. We also know that the street type "ST" is often an abbreviation for the street type "Street", and that sometimes suite types such as "Ste." are omitted from address representations. Taking these observations together, a human can quickly surmise with high confidence that these strings refer to the same address.

In order to teach a computer to do the same, the first step is to teach it to parse addresses into their components.

## Training data

The training data that we used to train the model was a list of tokenized addresses along with tags. Data was obtained using Amazon Mechanical Turk to have humans tag a list of 200 addresses. Of the 200, 185 were usable at the time of this writing. Here's a look at the first few entries in the training file to get a feel for what it looks like.

In [4]:
with open('data/tagged_addresses.json') as f:
    td = json.load(f)
    
print(json.dumps(td[0:3], indent = 2))

[
  {
    "raw_address": "612 S ALASKA ST",
    "tags": [
      "STREET_NUMBER",
      "PRE_DIRECTION",
      "STREET_NAME",
      "STREET_TYPE"
    ],
    "tokens": [
      "612",
      "S",
      "ALASKA",
      "ST"
    ]
  },
  {
    "raw_address": "540 RONLEE LN NW STE B",
    "tags": [
      "STREET_NUMBER",
      "STREET_NAME",
      "STREET_TYPE",
      "POST_DIRECTION",
      "UNIT_TYPE",
      "UNIT_NUMBER"
    ],
    "tokens": [
      "540",
      "RONLEE",
      "LN",
      "NW",
      "STE",
      "B"
    ]
  },
  {
    "raw_address": "624 SUNSET PARK DR",
    "tags": [
      "STREET_NUMBER",
      "STREET_NAME",
      "STREET_NAME",
      "STREET_TYPE"
    ],
    "tokens": [
      "624",
      "SUNSET",
      "PARK",
      "DR"
    ]
  }
]


Tags were permitted to be one of `UNIT_TYPE`, `UNIT_NUMBER`, `STREET_NUMBER`, `PRE_DIRECTION`, `STREET_NAME`, `STREET_TYPE`, or `POST_DIRECTION`.

## The CRF model

Linear chain conditional random field models are discriminative classifiers that allow classification of sequences of interdependent observations. It has been widely applied (see the link below) to _Named Entity Recognition_ tasks, which is the commonly used name for the general problem of applying labels to sequences of symbols--usually applying part-of-speech labels (Noun, Verb, etc.) to words.

An advantage of CRF for this particular task over other similar models such as the Hidden Markov Model (HMM) is that CRF can incorporate a wide variety of features aside from just the given order of tags from training data. This is because CRF models, being discriminative, model the conditional probability distribution $p(y|\mathbf{x})$ directly, without ever modeling the joint distribution $p(\mathbf{x},y)$. This is good news when the features $\mathbf{x}$ are high dimensional and highly interdependent, which is likely true in the case of many named entity recognition tasks.

(Source)
http://homepages.inf.ed.ac.uk/csutton/publications/crftutv2.pdf

### Feature Extraction

For the address parsing problem, an individual observation $y_j$ is a sequence $(l_1, l_2, ..., l_t)$ of $t$ labels. For each label $l_i$ in $y$, we also provide a sequence $(x_{i1}, x_{i2}, ..., x_{if})$ of features for that label, so the feature set is $$\mathbf{x}_j = ((x_{11}, x_{12}, ..., x_{1f}), ..., (x_{t1}, x_{t2}, ..., x_{tf}))$$

To bring this into practice, it was convenient to have a method to quickly obtain many features at once. To achieve this, (in perhaps a slight abuse of object orented programming methods) we created a class called a `FeatureFunctionApplicator`, which has a method `FeatureFunctionApplicator.exec_all(self, s)` which applies maps multiple feature functions to `s` and returns the results in a dictionary. Feature functions are recognized as functions starting with the prefix `f_`. An example follows.

In [5]:
class SillyFF(FeatureFunctionApplicator):
    def f_constant(self, s):
        return "constant"
    
    def f_length(self, s):
        return len(s)
    
    def f_first_letter(self, s):
        return s[0]

The above code makes a `FeatureFunctionApplicator` with two feature functions. One feature always returns the word "constant", and the other returns the length of the input. By instantiating this `FeatureFunctionApplicator`, we obtain a means of easily applying both of those functions 

In [6]:
sillyff = SillyFF()
sillyff.exec_all("word")

{'f_constant': 'constant', 'f_first_letter': 'w', 'f_length': 4}

This dictionary format with feature names as keys and feature values as values is what is expected by `pycrfsuite`. Using this class made it easy to experiment with a number of different configurations of features. After experimentation, the following feature set gave the best results.

In [7]:
class WordFeatures2(FeatureFunctionApplicator):

    def f_is_street_type_word(self, s: str):
        return s.lower().strip(punctuation) in STREET_TYPES

    def f_is_direction(self, s: str):
        return s.lower().strip(punctuation) in DIRECTIONS

    def f_is_digit1(self, s: str):
        return s.isdigit() and len(s) == 1

    def f_is_digit2(self, s: str):
        return s.isdigit() and len(s) == 2
    
    def f_is_digit3(self, s: str):
        return s.isdigit() and len(s) == 3
    
    def f_is_digit4(self, s: str):
        return s.isdigit() and len(s) == 4
    
    def f_is_digit5(self, s: str):
        return s.isdigit() and len(s) >= 5
    
    def f_has_digit(self, s):
        return re.search('\d', s) is not None

    def f_length(self, s: str):
        return len(s)
    
    def f_pound(self, s: str):
        return '#' in s
    
    def f_unit_type(self, s: str):
        return s.lower().strip(punctuation) in UNIT_TYPES
    
    def f_ends_in_hyphen(self, s:str):
        return s[-1] == '-'



## Training and Testing

The code below creates a `pycrfsuite.Trainer` object, which interfaces with the `crfsuite` application to create a trained model. We used a fairly large holdout set of 30% to evaluate the model performance.

In [8]:
seed(1729)
training_size = 0.7  # use 70% for training

trainer2 = pycrfsuite.Trainer()

group = []

with open('data/tagged_addresses.json') as f:
    td = json.load(f)

for item in td:
    g = int(uniform() < 0.7)
    features = [WordFeatures2().exec_all(t) for t in item['tokens']]
    if len(features) != len(item['tags']): print(item['tokens'])
    trainer2.append(xseq = features, yseq = item['tags'], group=g)
    group.append(g)

In [9]:
trainer2.train('model3', holdout = 0)

Holdout group: 1

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 103
Seconds required: 0.002

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 2147483647
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 1201.615388
Feature norm: 1.000000
Error norm: 966.266859
Active features: 103
Line search trials: 1
Line search step: 0.001164
Seconds required for this iteration: 0.000
Performance by label (#match, #model, #ref) (precision, recall, F1):
    STREET_NUMBER: (0, 0, 58) (0.0000, 0.0000, 0.0000)
    PRE_DIRECTION: (0, 0, 18) (0.0000, 0.0000, 0.0000)
    STREET_NAME: (70, 276, 70) (0.2536, 1.0000, 0.4046)
    STREET_TYPE: (0, 0, 59) (0.0000, 0.0000, 0.0000)
    POST_DIRECTION: (0, 0, 10) (0.0000, 0.0000, 0.0000)
    UNIT_TYPE: (0, 0, 28) (0.00

The model with these feature functions performs _extremely_ well, correctly tagging 56/58 training examples, or 272/276 training tags. Macro-averaged Precision, Recall, and F1 are all over 0.99.

In order to inspect the model further, we must create a `pycrfsuite.Tagger` object with the saved model (which is called `model3`). The tagger object has a model info method that we can use to inspect the transition probabilities and feature weights.

In [10]:
tagger3 = pycrfsuite.Tagger()
tagger3.open('model3')
model_info = tagger3.info()

The `Tagger.info` object is a little bit clunky to work with, but it contains a lot of information. The following code prints out the state transitions in order from highest to lowest conditional likelihood.

In [11]:
sorted([(v, k) for k, v in model_info.transitions.items()], reverse = True)

[(3.792563, ('UNIT_TYPE', 'UNIT_NUMBER')),
 (2.840407, ('STREET_NAME', 'STREET_TYPE')),
 (2.161069, ('STREET_NUMBER', 'PRE_DIRECTION')),
 (1.929822, ('STREET_NUMBER', 'STREET_NAME')),
 (1.727106, ('PRE_DIRECTION', 'STREET_NAME')),
 (1.553655, ('STREET_TYPE', 'POST_DIRECTION')),
 (1.295494, ('POST_DIRECTION', 'UNIT_TYPE')),
 (1.168599, ('STREET_NAME', 'STREET_NAME')),
 (0.959748, ('STREET_TYPE', 'UNIT_NUMBER')),
 (0.815372, ('STREET_TYPE', 'UNIT_TYPE')),
 (0.791657, ('UNIT_NUMBER', 'UNIT_NUMBER')),
 (0.49895, ('POST_DIRECTION', 'UNIT_NUMBER')),
 (0.388984, ('STREET_NAME', 'POST_DIRECTION')),
 (0.308312, ('PRE_DIRECTION', 'STREET_TYPE')),
 (0.205892, ('STREET_NUMBER', 'STREET_TYPE')),
 (-0.122769, ('UNIT_NUMBER', 'UNIT_TYPE')),
 (-0.179024, ('UNIT_NUMBER', 'STREET_NUMBER')),
 (-0.399301, ('STREET_TYPE', 'STREET_NAME')),
 (-0.626987, ('STREET_NUMBER', 'STREET_NUMBER'))]

And below are the feature weights for each tag.

In [12]:
sorted([(v, k) for k, v in model_info.state_features.items()], reverse=True)

[(2.752884, ('f_is_street_type_word', 'STREET_TYPE')),
 (2.348774, ('f_unit_type', 'UNIT_TYPE')),
 (2.100425, ('f_is_direction', 'POST_DIRECTION')),
 (1.814316, ('f_has_digit', 'STREET_NUMBER')),
 (1.793916, ('f_is_direction', 'PRE_DIRECTION')),
 (1.452739, ('f_is_digit4', 'STREET_NUMBER')),
 (1.429648, ('f_pound', 'UNIT_NUMBER')),
 (1.078772, ('f_is_digit5', 'STREET_NUMBER')),
 (1.019443, ('f_ends_in_hyphen', 'UNIT_NUMBER')),
 (0.928144, ('f_has_digit', 'UNIT_NUMBER')),
 (0.837322, ('f_is_digit3', 'STREET_NUMBER')),
 (0.442073, ('f_length', 'STREET_NAME')),
 (0.441718, ('f_is_digit1', 'UNIT_NUMBER')),
 (0.380232, ('f_is_digit2', 'STREET_NAME')),
 (0.224143, ('f_is_digit1', 'STREET_NAME')),
 (0.117415, ('f_length', 'UNIT_NUMBER')),
 (0.102074, ('f_is_digit3', 'UNIT_NUMBER')),
 (0.095591, ('f_length', 'STREET_NUMBER')),
 (0.069907, ('f_is_digit2', 'UNIT_NUMBER')),
 (0.023521, ('f_is_digit2', 'STREET_NUMBER')),
 (0.00925, ('f_length', 'UNIT_TYPE')),
 (-0.018462, ('f_ends_in_hyphen', 'PRE

ImportError: cannot import name 'AddressTagger'